In [6]:
from ray.tune.schedulers import PopulationBasedTrainingReplay

In [10]:
replay = PopulationBasedTrainingReplay(
    "./data/FashionMNIST-pbt/pbt_policy_898a5_00003.txt")

replay.config  # Initial config
  # Schedule, in the form of tuples (step, config)


{'progress_bar_refresh_rate': 0,
 'layer_1_size': 512,
 'layer_2_size': 512,
 'lr': 5e-05,
 'batch_size': 1024,
 'data_dir': './data',
 'data_mean': 0.28604063391685486,
 'data_std': 0.35302430391311646,
 'augmentations': [['blur', 0.26791761175640405],
  ['rotate_left', 0.8723674737797285],
  ['rotate_right', 0.07890809261122134]]}

In [11]:
replay._policy

[(8,
  {'progress_bar_refresh_rate': 0,
   'layer_1_size': 512,
   'layer_2_size': 512,
   'lr': 5e-05,
   'batch_size': 1024,
   'data_dir': './data',
   'data_mean': 0.28604063391685486,
   'data_std': 0.35302430391311646,
   'augmentations': [['blur', 0.7129817103713567],
    ['rotate_left', 0.09256880579677385],
    ['rotate_right', 0.13630303731298177]]}),
 (13,
  {'progress_bar_refresh_rate': 0,
   'layer_1_size': 512,
   'layer_2_size': 512,
   'lr': 5e-05,
   'batch_size': 1024,
   'data_dir': './data',
   'data_mean': 0.28604063391685486,
   'data_std': 0.35302430391311646,
   'augmentations': [['blur', 0.2172307474348526],
    ['rotate_left', 0.19635165171173796],
    ['rotate_right', 0.21067099883267293]]})]

In [12]:
from MNISTDataModule import MNISTDataModule
from augmentation import TRANSFORM_NAMES
from LightningFashionMNIST import LightningFashionMNIST
import pytorch_lightning as pl
import os
import torch

batch_and_lr_multiplier = 5
conf = {
    "progress_bar_refresh_rate": 25,
    "layer_1_size": 512,
    "layer_2_size": 512,
    "lr": 0.00001 * batch_and_lr_multiplier,
    "batch_size": 32 * (2 ** batch_and_lr_multiplier),
    "data_dir": "./data",
    # Fashion mnist mean and std
    "data_mean": 0.28604063391685486,
    "data_std": 0.35302430391311646,
    "augmentations": [],
}

model = LightningFashionMNIST.load_from_checkpoint(os.path.join("./data/FashionMNIST-pbt/train_mnist_tune_checkpoint_cc5a0_00002_2_2021-10-11_23-07-02/checkpoint_000014", "checkpoint"), conf=conf)
data = MNISTDataModule(conf=conf)

trainer = pl.Trainer(
    default_root_dir="./data",
    gpus=-1 if torch.cuda.device_count() > 0 else 0,
    max_epochs=15,
    progress_bar_refresh_rate=conf["progress_bar_refresh_rate"],
    num_sanity_val_steps=0,
    # callbacks=[early_stopping]
)


# test (pass in the model)
trainer.test(model, datamodule=data)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'avg_test_acc': 0.749707043170929,
 'avg_test_loss': 1.2627410888671875,
 'test_acc': 0.7497000098228455,
 'test_acc_epoch': 0.7497000098228455,
 'test_loss': 1.2631179094314575,
 'test_loss_epoch': 1.2631179094314575}
--------------------------------------------------------------------------------


[{'test_loss': 1.2631179094314575,
  'test_loss_epoch': 1.2631179094314575,
  'test_acc': 0.7497000098228455,
  'test_acc_epoch': 0.7497000098228455,
  'avg_test_loss': 1.2627410888671875,
  'avg_test_acc': 0.749707043170929}]